## This Notebook Cleans our Data

### This is quite a lot of data so it may take some time to complete depending on hardware resources

### I would recommend only having this notbook open and closing all other applications when running, unless you have good specs on desktop

#### Feel free to change the values to whatever you want if you are experementing with removing other data, like _std, _5_last_games, etc.


In [ ]:
import numpy as np
import pandas as pd
import os
import gc

In [ ]:
# Function to load and clean data
# We remove Standard deviation columns, and season average, and last 5 match sum statistics
def load_and_clean_data(file_path):
    """Load a dataset and apply cleaning steps.
    Removes STD, season average, and last 5 matched sum
    Fills NULL/nan values with 0/zero"""
    try:
        df = pd.read_csv(file_path)
        df = df.drop(df.filter(regex='_std$|_season_average$|5_last_match_sum$').columns, axis=1)

        # Fill remaining missing values with zero
        df = df.fillna(0)
        return df
    except FileNotFoundError as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [ ]:
# Function to reshape player data (convert multiple rows per ID into a single row)
def reshape_player_data(player_df):
    """Convert player stats from multiple rows to a single row per ID."""
    if player_df is None:
        return None

    # Add a unique number to each player's stats for a given ID
    player_df["player_number"] = player_df.groupby("ID").cumcount() + 1

    # Reshape using pivot_table (each player gets a numbered column)
    player_df = player_df.pivot(index="ID", columns="player_number")
    
    # Flatten MultiIndex columns
    player_df.columns = [f"{col[0]}_P{col[1]}" for col in player_df.columns]
    
    # Reset index so ID is a column again
    player_df = player_df.reset_index()
    
    return player_df

In [ ]:
# Function to load and clean data
# further cleaning the data after reshaping,
# removing Players 6-27, and TEAM_NAME, and Player_name, and League
def clean_data_V2(df):
    """Load a dataset and apply cleaning steps.
    remove P6-27
    and POSITION"""
    try:
        df = df.drop(df.filter(regex='P2[0-7]$|^POSITION|P1[0-9]$|^TEAM_NAME|^LEAGUE|^PLAYER_NAME|P[6-9]$').columns, axis=1)
        return df
    except FileNotFoundError as e:
        print(f"Error loading {df}: {e}")
        return None

In [ ]:
# Function to merge team and reshaped player data
# this function calls the other functions to clean the data
def merge_team_and_players(team_path, player_path, output_path):
    """Merge a team dataset with its players into a single row per ID and save to file."""
    print(f"Processing and saving: {output_path}")

    # Load and clean data
    team_df = load_and_clean_data(team_path)
    player_df = load_and_clean_data(player_path)

    if team_df is None or player_df is None:
        print(f"Skipping {output_path} due to missing data.")
        return

    # Reshape player stats
    reshaped_players = reshape_player_data(player_df)

    # Merge team stats with reshaped player stats (1 row per ID)
    merged_df = pd.merge(team_df, reshaped_players, on='ID', how='left')
    
    cleaned_v2 = clean_data_V2(merged_df)

    # Save final dataset
    cleaned_v2.to_csv(output_path, index=False)
    print(f"Saved {output_path} ({cleaned_v2.shape[0]} rows, {cleaned_v2.shape[1]} columns)")

    # Free up memory
    del team_df, player_df, reshaped_players, merged_df, cleaned_v2
    gc.collect()
    return None
    # return team_df, reshaped_players, cleaned_v2

In [ ]:
# Define dataset paths
# Put the path to your UNCLEAN data
data_paths = {
    "train_home_team": "C:/Path/To/Data/Train_Data/train_home_team_statistics_df.csv",
    "train_home_player": "C:/Path/To/Data/Train_Data/train_home_player_statistics_df.csv",
    "train_away_team": "C:/Path/To/Data/Train_Data/train_away_team_statistics_df.csv",
    "train_away_player": "C:/Path/To/Data/Train_Data/train_away_player_statistics_df.csv",
    "test_home_team": "C:/Path/To/Data/Test_Data/test_home_team_statistics_df.csv",
    "test_home_player": "C:/Path/To/Data/Test_Data/test_home_player_statistics_df.csv",
    "test_away_team": "C:/Path/To/Data/Test_Data/test_away_team_statistics_df.csv",
    "test_away_player": "C:/Path/To/Data/Test_Data/test_away_player_statistics_df.csv"
}

In [ ]:
# CLeaning Columns
# Create output directory
output_dir = "C:/Path/To/Data/cleanedData/TestingCleanedDataGroupTest"
os.makedirs(output_dir, exist_ok=True)

# Merge and save Home datasets separately
# teamDF, reshapedDATA, cLEANV2 = merge_team_and_players(data_paths["train_home_team"], data_paths["train_home_player"], os.path.join(output_dir, "train_merged_home.csv"))
merge_team_and_players(data_paths["train_home_team"], data_paths["train_home_player"], os.path.join(output_dir, "train_home.csv"))
merge_team_and_players(data_paths["test_home_team"], data_paths["test_home_player"], os.path.join(output_dir, "test_home.csv"))

# # Merge and save Away datasets separately
merge_team_and_players(data_paths["train_away_team"], data_paths["train_away_player"], os.path.join(output_dir, "train_away.csv"))
merge_team_and_players(data_paths["test_away_team"], data_paths["test_away_player"], os.path.join(output_dir, "test_away.csv"))

print("All datasets merged and saved successfully!")

##### Looking at the size diference at different stages of cleaning

In [ ]:
print(teamDF.shape)
# print(teamDF.head)
print(reshapedDATA.shape)
# print(reshapedDATA.head)
print(cLEANV2.shape)
# print(cLEANV2.head)
print(cLEANV2.columns)

## Combining Away and Home data

In [ ]:

X_train_home = pd.read_csv('C:/Path/You?Want?Data/To/train_home.csv')
X_train_away = pd.read_csv('C:/Path/You?Want?Data/To/train_away.csv')

X_test_home = pd.read_csv('C:/Path/You?Want?Data/To/test_home.csv')
X_test_away = pd.read_csv('C:/Path/You?Want?Data/To/test_away.csv')

X_train_combined = pd.concat([X_train_home, X_train_away], axis=1)
X_test_combined = pd.concat([X_test_home, X_test_away], axis=1)


X_train_combined.to_csv(f"C:/Path/You?Want?Data/To/trainV5.1_combine.csv", index=False)
X_test_combined.to_csv(f"C:/Path/You?Want?Data/To/testV5.1_combine.csv", index=False)




# No Longer Needed


### Ensuring that Y and X have same ID values

In the function below, Give the path to your cleaned data. 
Just use your output paths from above

In [ ]:
Y_data = "Path/to/cleaned/data"
home ="Path/to/cleaned/data"
away = "Path/to/cleaned/data"


def align_training_datasets(Y_train, X_train_home, X_train_away, output_path_prefix):
    """
    Align Y_train, X_train_home, and X_train_away datasets to ensure they have the same IDs
    and number of rows. Also checks for duplicates and validates data integrity.
    
    Parameters:
    -----------
    Y_train : pandas.DataFrame
        Training labels dataset containing IDs
    X_train_home : pandas.DataFrame
        Home team training features dataset containing IDs
    X_train_away : pandas.DataFrame
        Away team training features dataset containing IDs
    output_path_prefix : str
        Prefix for the output files. Will append _y.csv, _home.csv, and _away.csv
        
    Returns:
    --------
    tuple
        (aligned_y_train, aligned_x_train_home, aligned_x_train_away)
    """
    # Convert IDs to string type for consistent comparison
    Y_train['ID'] = Y_train['ID'].astype(str)
    X_train_home['ID'] = X_train_home['ID'].astype(str)
    X_train_away['ID'] = X_train_away['ID'].astype(str)
    
    # Check for duplicate IDs in each dataset
    print("Checking for duplicates:")
    print(f"Y_train duplicates: {Y_train['ID'].duplicated().sum()}")
    print(f"X_train_home duplicates: {X_train_home['ID'].duplicated().sum()}")
    print(f"X_train_away duplicates: {X_train_away['ID'].duplicated().sum()}")
    
    # Get sets of IDs from each dataset
    y_ids = set(Y_train['ID'])
    home_ids = set(X_train_home['ID'])
    away_ids = set(X_train_away['ID'])
    
    # Find common IDs across all datasets
    common_ids = y_ids.intersection(home_ids).intersection(away_ids)
    
    # Print analysis of missing IDs
    print("/nMissing ID Analysis:")
    print(f"IDs in Y_train but not in X_train_home: {y_ids - home_ids}")
    print(f"IDs in Y_train but not in X_train_away: {y_ids - away_ids}")
    print(f"IDs in X_train_home but not in Y_train: {home_ids - y_ids}")
    print(f"IDs in X_train_away but not in Y_train: {away_ids - y_ids}")
    
    # Filter all datasets to keep only common IDs
    aligned_y = Y_train[Y_train['ID'].isin(common_ids)].copy()
    aligned_home = X_train_home[X_train_home['ID'].isin(common_ids)].copy()
    aligned_away = X_train_away[X_train_away['ID'].isin(common_ids)].copy()
    
    # Sort all datasets by ID to ensure matching order
    aligned_y = aligned_y.sort_values('ID').reset_index(drop=True)
    aligned_home = aligned_home.sort_values('ID').reset_index(drop=True)
    aligned_away = aligned_away.sort_values('ID').reset_index(drop=True)
    
    # Print shape information
    print("/nDataset Shapes:")
    print(f"Original Y_train shape: {Y_train.shape}")
    print(f"Original X_train_home shape: {X_train_home.shape}")
    print(f"Original X_train_away shape: {X_train_away.shape}")
    print(f"/nAligned Y_train shape: {aligned_y.shape}")
    print(f"Aligned X_train_home shape: {aligned_home.shape}")
    print(f"Aligned X_train_away shape: {aligned_away.shape}")
    
    # Verify alignment
    all_aligned = (
        aligned_y.shape[0] == aligned_home.shape[0] == aligned_away.shape[0] and
        (aligned_y['ID'] == aligned_home['ID']).all() and
        (aligned_y['ID'] == aligned_away['ID']).all()
    )
    print(f"/nDatasets properly aligned: {all_aligned}")
    
    # Save aligned datasets
    if output_path_prefix:
        aligned_y.to_csv(f"{output_path_prefix}_y.csv", index=False)
        aligned_home.to_csv(f"{output_path_prefix}_home.csv", index=False)
        aligned_away.to_csv(f"{output_path_prefix}_away.csv", index=False)
        print(f"/nAligned datasets saved with prefix: {output_path_prefix}")

    
# Example usage
align_training_datasets(Y_data, home, away, 'C:/Path/To/Data/Test_Data/DataV4.1_test')